# 6.7 Preprocessing Data for Tableau Dashboard: Geo Shape Polygons

### This script contains the following:
#### 1. Import data and libraries
#### 2. Geo Shape - Polygon

### 1. Import data and libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import os
import folium
import json
from shapely.geometry import shape, Point

In [2]:
# This command propts matplotlib visuals to appear in the notebook 

%matplotlib inline

In [3]:
path = r'/Users/kristincpeterson/Desktop/DataAnalytics/Achievement_6/Final Project - Espaces Verts'

In [4]:
df = pd.read_csv(os.path.join(path, '02 Data', 'Prepared_Data', 'espaces_verts_75_updated.csv'))

In [5]:
df.shape

(2286, 25)

In [6]:
df.head()

,Unnamed: 0.1,Unnamed: 0,ID,Name,Type,Category,Street Number,Street Number Suffix,Street Type,Street Name,...,Perimeter,Year opened,Year renovated,Former name,Year name changed,Number of entities,Open 24hrs,Geo Shape,URL_PLAN,Area category
0,0,0,11255.0,JARDINIERES DU 39 QUAI DE LA SEINE,Décorations sur la voie publique,Jardiniere,39.0,NaN,QUAI DE LA,SEINE,...,30.737389,2011.0,NaN,NaN,NaN,2.0,NaN,"{""coordinates"": [[[[2.3733184661747395, 48.886...",http://p70-planet.apps.paris.mdp/Advitium/Appe...,Small size
1,1,1,10804.0,JARDINIERES DU PARVIS DE LA PLACE RAOUL DAUTRY,Décorations sur la voie publique,Jardiniere,6.0,S,PLACE,RAOUL DAUTRY,...,312.718568,NaN,NaN,NaN,NaN,4.0,NaN,"{""coordinates"": [[[[2.3216385869638, 48.841417...",http://p70-planet.apps.paris.mdp/Advitium/Appe...,Large size
2,2,2,10937.0,JARDINIERES ANGLE DES RUES DE TOLBIAC - BAUDRI...,Décorations sur la voie publique,Jardiniere,47.0,NaN,RUE,BAUDRICOURT,...,80.353023,2005.0,NaN,NaN,NaN,2.0,NaN,"{""coordinates"": [[[[2.363882470474931, 48.8265...",http://p70-planet.apps.paris.mdp/Advitium/Appe...,Small size
3,3,3,219.0,JARDIN FRANCOISE GIROUD,Promenades ouvertes,Jardin,4.0,V,PLACE D,ITALIE,...,245.468867,1878.0,2019.0,SQUARE DE LA PLACE D'ITALIE,2013.0,1.0,Oui,"{""coordinates"": [[[2.356129760906164, 48.83152...",http://p70-planet.apps.paris.mdp/Advitium/Appe...,Very large size
4,4,4,10303.0,JARDINIERE DE LA PLACE FRANZ LISZT,Décorations sur la voie publique,Jardiniere,2.0,BIS V,PLACE,FRANZ LISTZ,...,52.677758,NaN,NaN,NaN,NaN,1.0,NaN,"{""coordinates"": [[[2.351449405515269, 48.87773...",http://p70-planet.apps.paris.mdp/Advitium/Appe...,Small size


In [7]:
# Remove Unnamed columns

df.drop(columns=['Unnamed: 0', 'Unnamed: 0.1'], inplace=True)

In [8]:
# Check missing values 
missing_values = df.isnull().sum()

# Calculate the percentage of missing values for each column
missing_percentage = (missing_values / len(df)) * 100

# Display the percentage of missing values for each column
missing_percentage.sort_values(ascending=False)

Year renovated          95.756780
Year name changed       87.620297
Street Number Suffix    84.033246
Former name             77.909011
Open 24hrs              62.948381
Year opened             31.758530
Horticultural area      25.940507
Perimeter               24.890639
Actual total area       23.972003
Area category           23.972003
Calculated area         13.604549
Street Name             10.936133
Street Type             10.804899
Number of entities       3.412073
URL_PLAN                 2.755906
ID                       2.755906
Enclosure                2.143482
Street Number            0.087489
Geo Shape                0.087489
Type                     0.043745
Name                     0.000000
Zip code                 0.000000
Category                 0.000000
dtype: float64

In [9]:
df.dtypes

ID                      float64
Name                     object
Type                     object
Category                 object
Street Number           float64
Street Number Suffix     object
Street Type              object
Street Name              object
Zip code                float64
Calculated area         float64
Actual total area       float64
Horticultural area      float64
Enclosure                object
Perimeter               float64
Year opened             float64
Year renovated          float64
Former name              object
Year name changed       float64
Number of entities      float64
Open 24hrs               object
Geo Shape                object
URL_PLAN                 object
Area category            object
dtype: object

### 2. Geo Shape - Polygon

#### Prepare data to visualize complex shapes like polygons or lines from GeoJSON data in Tableau. 
#### The goal here is to convert each GeoJSON shape into a series of points that Tableau can understand and then use these points to draw the shapes.

In [10]:
# Ensure 'Geo Shape' is a string and not missing

df = df.dropna(subset=['Geo Shape'])
df['Geo Shape'] = df['Geo Shape'].astype(str)

In [13]:
# Convert GeoJSON shapes to latitude and longitude points

def extract_points(geojson_str):
    """
    Extracts all points from a GeoJSON polygon and returns them as a list of tuples
    (longitude, latitude, order).
    """
    points = []
    try:
        # Convert the GeoJSON string into a Python dictionary
        geojson = json.loads(geojson_str)
        
        # Use Shapely to create a shape
        shp = shape(geojson)
        
        # Use geom_type instead of type
        if shp.geom_type == 'Polygon':
            exterior_coords = shp.exterior.coords
            points += [(x, y, order) for order, (x, y) in enumerate(exterior_coords)]
        
        # Correct iteration over MultiPolygon
        elif shp.geom_type == 'MultiPolygon':
            for polygon in shp.geoms:
                exterior_coords = polygon.exterior.coords
                points += [(x, y, order) for order, (x, y) in enumerate(exterior_coords)]
                
    except Exception as e:
        print(f"Error processing GeoJSON: {e}")
    
    return points

# Apply the function to each row in the DataFrame and expand the list of points into rows
all_points = []
for index, row in df.iterrows():
    points = extract_points(row['Geo Shape'])
    for point in points:
        all_points.append((row['ID'], *point))

# Convert the list of points into a DataFrame
points_df = pd.DataFrame(all_points, columns=['ID', 'Longitude', 'Latitude', 'Order'])


In [14]:
# Exporting the points for use in Tableau

points_df.to_csv(os.path.join(path, '02 Data','Prepared_Data', 'processed_points.csv'))

#### While this method technically worked, the polygon shapes that Tableau produced are not very pretty, and the map is harder to read. Unfortunately, Tableau does not seem to be the best tool to visualize these polygon shapes. Instead, we will use a single point on the map to represent each green space in Tableau.